(rotazione-notebook)=
# La rotazione fattoriale 

In [1]:
source("_common.R")
suppressPackageStartupMessages({
    library("lavaan")
    library("psych")
})
options(repr.plot.width = 6, repr.plot.height = 6)
set.seed(42)


Nel capitolo {ref}`extraction-notebook` abbiamo visto come sia possibile ottenere la
soluzione fattoriale non ruotata per il numero di fattori comuni che
meglio riassume l'informazione contenuta nella matrice di correlazioni
(o covarianze). Tuttavia, questa soluzione non garantisce l’identificazione di gruppi omogenei e interpretabili di variabili osservate. Pertanto, si ricorre alla rotazione degli assi fattoriali per ottenere una soluzione più facilmente interpretabile rispetto a quella iniziale.


## Indeterminatezza della soluzione fattoriale

La necessità di effettuare la rotazione deriva dal fatto che la matrice delle saturazioni non ha un’unica soluzione. Attraverso trasformazioni matematiche, è possibile ottenere infinite matrici dello stesso ordine. Questo fenomeno è noto come indeterminatezza della soluzione fattoriale.

La matrice delle saturazioni fattoriali $\boldsymbol{\Lambda}$ non è univocamente definita poiché non esiste una soluzione unica per determinare le saturazioni fattoriali. Una matrice di correlazioni $\boldsymbol{R}$ può produrre diverse soluzioni fattoriali, ovvero matrici con lo stesso numero di fattori comuni ma con diverse configurazioni di saturazioni fattoriali, o matrici di saturazioni fattoriali corrispondenti a un diverso numero di fattori comuni.

**Esempio.** Siano $\boldsymbol{\Lambda}_1$ e $\boldsymbol{\Lambda}_2$ due matrici
aventi lo stesso numero di righe e colonne, ma contenenti saturazioni
fattoriali diverse. $\boldsymbol{\Lambda}_1$ è definita dai valori
seguenti

In [ ]:
l1 <- matrix(
  c(
    0.766,  -0.232,
    0.670,  -0.203,
    0.574,  -0.174,
    0.454,   0.533,
    0.389,   0.457,
    0.324,   0.381
  ),
  byrow = TRUE, ncol = 2
)


mentre per $\boldsymbol{\Lambda}_2$ abbiamo

In [ ]:
l2 <- matrix(
  c(
    0.783,  0.163,
    0.685,  0.143,
    0.587,  0.123,
    0.143,  0.685,
    0.123,  0.587,
    0.102,  0.489
  ),
  byrow = TRUE, ncol = 2
)


Esaminiamo la matrice delle correlazioni riprodotte dalle due matrici di
pesi fattoriali (con le comunalità sulla diagonale di $\boldsymbol{R}$):

In [ ]:
l1 %*% t(l1)

In [ ]:
l2 %*% t(l2)

Come si vede, viene ottenuto lo stesso risultato utilizzando matrici
$\boldsymbol{\Lambda}$ con lo stesso numero $m$ di colonne ma
saturazioni fattoriali diverse.

Si consideri ora il caso di matrici $\boldsymbol{\Lambda}$
corrispondenti a soluzioni fattoriali con un diverso numero di fattori
comuni. Siano $\boldsymbol{\Lambda}_1$ e $\boldsymbol{\Lambda}_2$ due
matrici aventi lo stesso numero di righe ma un numero diverso di
colonne:

In [ ]:
l1 <- matrix(
  c(
    0.9,
    0.7,
    0.5,
    0.3
  ),
  byrow = TRUE, ncol = 1
)

l2 <- matrix(
  c(
    0.78, 0.45,
    0.61, 0.35,
    0.43, 0.25,
    0.25, 0.15
  ),
  byrow = TRUE, ncol = 2
)


Si noti che la stessa matrice di correlazioni riprodotte (con le
comunalità sulla diagonale principale) viene generata dalle saturazioni
fattoriali corrispondenti ad un numero diverso di fattori comuni:

In [ ]:
l1 %*% t(l1)

In [ ]:
l2 %*% t(l2)

## Parsimonia e semplicità

Per ottenere risultati affidabili dall'analisi fattoriale, si affronta il problema dell'indeterminazione fattoriale scegliendo la soluzione che soddisfa due criteri fondamentali: il criterio della parsimonia e il criterio della semplicità.

Il criterio della parsimonia richiede di scegliere il modello con il minor numero di fattori comuni che può spiegare la covarianza tra le variabili. In pratica, se ci sono due soluzioni fattoriali con un diverso numero di fattori che riproducono allo stesso modo la matrice di covarianza o di correlazione, si sceglie quella con il minor numero di fattori.

In caso invece ci siano diverse soluzioni fattoriali con lo stesso numero m di fattori, il criterio della semplicità guida nella scelta della trasformazione più appropriata della matrice di saturazioni fattoriali $\boldsymbol{\Lambda}$. Questa trasformazione, nota come rotazione, cerca di rendere i fattori più interpretabili. Ci sono due tipi di rotazione: ortogonale e obliqua. 

La rotazione ortogonale assume che i fattori siano incorrelati, mentre la rotazione obliqua consente correlazioni tra i fattori. L'obiettivo della rotazione è di trovare una soluzione che renda i fattori più facilmente interpretabili e, quindi, in grado di spiegare meglio i dati.

### Il criterio della struttura semplice

La rotazione degli assi fattoriali è un metodo utilizzato nell'analisi fattoriale per ottenere una struttura semplice della matrice delle saturazioni fattoriali. Questa struttura si caratterizza per poche ma forti saturazioni diverse da zero e dall'assenza di variabili saturate da più di un fattore. Thurstone (1947) propose un criterio di struttura semplice che si raggiunge quando si soddisfano alcune condizioni nella matrice fattoriale ruotata, come la presenza di saturazioni nulle per ogni variabile e per ogni fattore almeno $m$ saturazioni nulle, dove $m$ è il numero di fattori comuni. 

In pratica, si cerca di massimizzare il numero di saturazioni nulle o quasi nulle nella matrice fattoriale ruotata per ottenere la struttura semplice e facilitare l'interpretazione dei fattori. L'esame delle saturazioni fattoriali nella matrice ruotata consente di fornire un'interpretazione ai fattori, osservando quali variabili sono maggiormente associate con essi e quanto forti sono questi legami. Se i coefficienti di impatto di un fattore sono positivi e elevati su un sottoinsieme di variabili osservate, deduciamo che il fattore rappresenta ciò che hanno in comune le variabili che saturano sul fattore. Tuttavia, l'interpretazione può diventare più complessa nel caso di variabili che saturano su più fattori.


## Rotazione nello spazio geometrico

### Rotazione ortogonale

Come abbiamo visto in precedenza, la matrice delle saturazioni fattoriali non è identificabile, il che significa che esistono molteplici soluzioni equivalenti per la determinazione dei pesi fattoriali. La rotazione ortogonale consiste nell'applicare una trasformazione lineare ai pesi fattoriali, in modo tale che venga prodotta una nuova matrice di saturazioni fattoriali che soddisfa alcuni criteri di struttura semplice, come descritto in precedenza. In altre parole, l'obiettivo della rotazione ortogonale è quello di ottenere una rappresentazione più interpretabile dei dati.

Geometricamente, la rotazione ortogonale corrisponde a una rotazione rigida degli assi cartesiani del sistema di coordinate che rappresenta i pesi fattoriali. Questa rotazione mantiene invariata la distanza tra i punti che rappresentano le saturazioni fattoriali, ma cambia la posizione relativa delle variabili rispetto ai fattori. In questo modo, è possibile ottenere una struttura dei pesi fattoriali più semplice e interpretabile. La rotazione ortogonale viene solitamente effettuata utilizzando la tecnica della massima verosimiglianza o del metodo dei componenti principali, al fine di massimizzare il numero di saturazioni nulle o quasi nulle nella nuova matrice delle saturazioni fattoriali.

### Vincoli alla rotazione

Il problema della non identificabilità di $\hat{\boldsymbol{\Lambda}}$ significa che ci sono molte possibili matrici di pesi fattoriali che possono produrre gli stessi valori di correlazione tra le variabili del nostro modello. Per risolvere questo problema, è necessario impostare dei vincoli sulla rotazione dei fattori. Il criterio principale nella scelta della rotazione è quello della semplicità della matrice $\hat{\boldsymbol{\Lambda}}$, ovvero la vicinanza dei suoi elementi ai valori 0 e 1. Questo rende più facile interpretare i fattori come combinazioni lineari delle variabili.

Le rotazioni ortogonali non cambiano le comunalità nel caso di fattori incorrelati, perché preservano le distanze tra i punti identificati dai pesi fattoriali. Questo significa che la comunalità può essere calcolata come la somma dei quadrati dei pesi fattoriali. Tuttavia, le rotazioni non ortogonali cambiano la quota di varianza spiegata da ciascun fattore, perché la varianza spiegata è data dalla somma dei quadrati dei pesi fattoriali divisa per la traccia della matrice delle correlazioni.

Ci sono diversi algoritmi per la rotazione ortogonale dei fattori. Alcuni di essi sono il metodo grafico, il metodo Quartimax e il metodo Varimax.

### Metodo grafico

Per ruotare il sistema degli assi, se ci sono solo $m=2$ fattori, utilizziamo un sistema di coordinate bidimensionale per ottenere la loro rappresentazione geometrica. L'ispezione visiva del diagramma delle saturazioni fattoriali può guidare nella scelta della rotazione più appropriata. Le righe di $\hat{\boldsymbol{\Lambda}}$ corrispondono a coppie di pesi fattoriali, $\hat{\lambda}_{i1}, \hat{\lambda}_{i2}$, con $i=1, \dots, p$, che possono essere interpretate come le coordinate di $p$ punti (tanti quanti le variabili manifeste). Gli assi del diagramma vengono ruotati di un angolo $\phi$ per avvicinarli il più possibile ai punti presenti nel grafico. Le nuove coordinate ($\hat{\lambda}_{i1}^*, \hat{\lambda}_{i2}^*$) vengono calcolate come $\hat{\boldsymbol{\Lambda}}^* = \hat{\boldsymbol{\Lambda}} \textbf{T}$,
dove 

$$
\textbf{T} = 
\left[
  \begin{array}{ c c }
  \cos{\phi} & - \sin{\phi}\\
  \sin{\phi} & \cos{\phi}
  \end{array} 
\right] 
$$ 

è una matrice ortogonale $2 \times 2$.

**Esempio.** Si considerino i dati di Brown, Williams e Barlow (1984) discussi da
{cite:t}`rencher10methods`. Ad una bambina di dodici anni è stato chiesto di
valutare sette dei suoi conoscenti su cinque variabili: *kind*,
*intelligent*, *happy*, *likeable* e *just*. Per queste cinque
variabili, la matrice di correlazioni è

In [1]:
R <- matrix(
  c(
    1.00, .296, .881, .995, .545,
    .296, 1.000, -.022, .326, .837,
    .881, -.022, 1.000, .867, .130,
    .995, .326, .867, 1.000, .544,
    .545, .837, .130, .544, 1.00
  ),
  ncol = 5, byrow = TRUE, dimnames = list(
    c("K", "I", "H", "L", "J"), c("K", "I", "H", "L", "J")
  )
)

print(R)

      K      I      H     L     J
K 1.000  0.296  0.881 0.995 0.545
I 0.296  1.000 -0.022 0.326 0.837
H 0.881 -0.022  1.000 0.867 0.130
L 0.995  0.326  0.867 1.000 0.544
J 0.545  0.837  0.130 0.544 1.000


Dalla matrice **R** estraiamo due fattori con il metodo delle componenti
principali:

In [ ]:
f.pc <- principal(R, 2, rotate = FALSE) 
f.pc

Si noti che i fattori sono difficili da interpretare: il primo fattore è caratterizzato da saturazioni fattoriali alte e positive su tutte le variabili manifeste; il secondo fattore è identificato dalle cinque variabili mediante una saturazione fattoriale alta e positiva, mentre le altre saturazioni fattoriali sono negative.

Nella seguente figura, i punti rappresentano le cinque coppie di saturazioni
fattoriali non ruotate (una per ciascun fattore):

In [ ]:
plot(
  f.pc$load[, 1], f.pc$load[, 2],
  bty = "n", xaxt = "n",
  xlab = "Primo Fattore", ylab = "Secondo Fattore",
  ylim = c(-.6, 1), xlim = c(0, 1), pch = 19, asp = 1
)
axis(1, pos = c(0, 0))
abline(0, 0)


{cite:t}`rencher10methods` nota che, per questi dati, una rotazione ortogonale di
$-35^{\circ}$ porterebbe gli assi ad avvicinarsi ai punti del
diagramma a dispersione. Per verificare questo, disegnamo sul diagramma i nuovi assi
dopo una rotazione di $-35^{\circ}$. Le istruzioni `R` sono le seguenti:

In [ ]:
plot(
  f.pc$load[, 1], f.pc$load[, 2],
  bty = "n", xaxt = "n",
  xlab = "Primo Fattore", ylab = "Secondo Fattore",
  ylim = c(-.6, 1), xlim = c(0, 1), pch = 19, asp = 1
)
axis(1, pos = c(0, 0))
abline(0, 0)

ar <- matrix(c(
  0, 0,
  0, 1,
  0, 0,
  1, 0
), ncol = 2, byrow = TRUE)

angle <- 35
rad <- angle * pi / 180
T <- matrix(c(
  cos(rad), -sin(rad),
  sin(rad),  cos(rad)
), ncol = 2, byrow = TRUE)

round(ar %*% T, 3)

arrows(0, 0, 0.574, 0.819, lwd = 2)
arrows(0, 0, 0.819, -0.574, lwd = 2)


Nella figura, le due frecce rappresentano gli assi ruotati. La rotazione di $-35^{\circ}$ ha effettivamente avvicinato gli assi ai punti del diagramma. Se usiamo dunque il valore $\phi = -35^{\circ}$ nella matrice di rotazione, possiamo
calcolare le saturazioni fattoriali della soluzione ruotata
$\hat{\boldsymbol{\Lambda}}^* = \hat{\boldsymbol{\Lambda}} \textbf{T}$.

Le saturazioni fattoriali ruotate corrispondono alla proiezione
ortogonale dei punti sugli assi ruotati:

In [ ]:
angle <- -35
rad <- angle * pi / 180
T <- matrix(c(
  cos(rad), -sin(rad),
  sin(rad),  cos(rad)
), ncol = 2, byrow = TRUE)
round(f.pc$load %*% T, 3)


La soluzione ottenuta in questo modo riproduce quanto riportato da
{cite:t}`rencher10methods`.


### Medodi di rotazione ortogonale

Un tipo di rotazione ortogonale spesso utilizzata è la rotazione Varimax
(Kaiser, 1958). La matrice $\hat{\boldsymbol{\Lambda}}$ è semplificata
in modo tale che le varianze dei quadrati degli elementi $\lambda_{ij}$
appartenenti a colonne diverse di $\hat{\boldsymbol{\Lambda}}$ siano
massime. Se le saturazioni fattoriali in una colonna di
$\hat{\boldsymbol{\Lambda}}$ sono simili tra loro, la varianza sarà
prossima a zero. Tale varianza è tanto più grande quanto più i quadrati
degli elementi $\lambda_{ij}$ assumono valori prossimi a $0$ e $1$.
Amplificando le correlazioni più alte e riducendo quelle più basse, la
rotazione Varimax agevola l'interpretazione di ciascun fattore.

Usando la funzione `factanal()` del modulo R base, la rotazione Varimax
può essere applicata alla soluzione ottenuta mediante il metodo di
massima verosimiglianza. Usando le funzioni `principal()` e
`factor.pa()` disponibili nel pacchetto `psych`, la rotazione Varimax
può essere applicata alle soluzioni ottenute mediante il metodo delle
componenti principali e il metodo del fattore principale. 

Ad esempio, usando il metodo delle componenti principali otteniamo:

In [ ]:
f_pc <- principal(R, 2, n.obs = 7, rotate = "varimax")
f_pc

Un altro metodo di rotazione ortogonale è il metodo Quartimax (Neuhaus e Wringley, 1954), il quale opera una semplificazione della matrice $\hat{\boldsymbol{\Lambda}}$ massimizzando le covarianze tra i quadrati degli elementi $\lambda_{ij}$ appartenenti a righe diverse, subordinatamente alla condizione che la varianza delle righe
rimanga inalterata.

### Metodi di rotazione obliqua

Il termine “rotazione obliqua” è improprio poiché una rotazione implica una trasformazione ortogonale che preserva le distanze. Secondo {cite:t}`rencher10methods`, un termine più appropriato sarebbe “trasformazione obliqua”. Tuttavia, il termine “rotazione obliqua” è di uso comune.

Nella rotazione obliqua, gli assi della soluzione ruotata non devono rimanere ortogonali e quindi possono avvicinarsi più facilmente ai raggruppamenti di punti nello spazio delle saturazioni fattoriali (se esistono). Esistono vari metodi analitici per ottenere una rotazione obliqua. Per esempio, nel metodo Direct Oblimin (Jennrich e Sampson, 1966), il criterio usato è
il seguente:

$$
\sum_{ij} \left(\sum_v \lambda_i^2 \lambda_j^2 - w \frac{1}{p} \sum_v \lambda_i^2
\sum_v \lambda_j^2\right)
$$ 

dove $\sum_{ij}$ si riferisce alla somma su tutte le coppie di fattori $ij$. In questo caso si procede ad una minimizzazione piuttosto che a una masssimizzazione.

## Matrice dei pesi fattoriali e matrice di struttura

Nella rotazione ortogonale i fattori sono incorrelati. Si consideri la situazione presentata nella figura successiva, con due variabili latenti incorrelate
($\xi_1$ e $\xi_2$) e quattro variabili manifeste ($y_1$, $y_2$, $y_3$,
$y_4$). Siano $\lambda_{11}$, $\lambda_{12}$, $\lambda_{13}$ e
$\lambda_{14}$ le saturazioni fattoriali delle variabili nel primo
fattore; siano $\lambda_{21}$, $\lambda_{22}$, $\lambda_{23}$ e
$\lambda_{24}$ le saturazioni fattoriali delle variabili nel secondo
fattore.

```{figure} images/rot_4.png
---
height: 300px
name: orto-rot-fig
---
Rotazione ortogonale.
```

In un diagramma di percorso, la correlazione tra due variabili è uguale alla somma dei valori numerici di tutti i percorsi legittimi che le collegano. Se i fattori comuni sono incorrelati, c’è un unico percorso legittimo che collega ciascuna variabile manifesta a ciascun fattore comune in base alle regole di Wright. Le correlazioni tra variabili manifeste e fattori comuni sono quindi uguali alle saturazioni fattoriali. Nel caso di fattori comuni incorrelati, dunque, la matrice delle saturazioni fattoriali descrive le correlazioni tra variabili e fattori. Le saturazioni fattoriali possono essere interpretate come i pesi beta del modello di regressione multipla, ovvero come la stima del contributo specifico di ciascun fattore comune nella determinazione della varianza spiegata degli item (Tabachnick & Fidell, 2001).

Nel caso della rotazione obliqua, la soluzione fattoriale ruotata produce un insieme di fattori comuni correlati tra loro. Di conseguenza, la matrice delle saturazioni fattoriali non descrive le correlazioni tra variabili e fattori. In un diagramma di percorso ci sono almeno due percorsi legittimi che collegano ciascuna variabile manifesta a ciascun fattore comune in base alle regole di Wright. Nel caso di una rotazione obliqua è quindi necessario specificare tre diverse matrici:

-   la matrice delle saturazioni fattoriali, $\hat{\boldsymbol{\Lambda}}$, detta
    *matrice pattern* (*factor pattern matrix*, o "configurazione," o
    "matrice dei modelli");
-   la matrice delle correlazioni tra variabili manifeste e fattori, detta *matrice di struttura* (*factor structure matrix*);
-   la  matrice che esprime le correlazioni tra i fattori, $\hat{\boldsymbol{\Phi}}$, detta *matrice di intercorrelazione fattoriale*.

In questo caso, la matrice pattern rappresenta l'analogo dei coefficienti parziali di regressione della variabile sul fattore, al netto degli altri fattori. Nel caso
della rotazione obliqua, è la matrice che viene usata per determinare in
che grado è stata raggiunta la "struttura semplice".

Esaminiamo in dettaglio la soluzione fattoriale prodotta da una rotazione obliqua. In questo caso, gli assi che rappresentano i fattori non sono ortogonali (ovvero, i fattori sono correlati) e, in un diagramma di percorso, le variabili manifeste sono collegate ai fattori attraverso due tipi distinti di percorsi che rappresentano l'effetto "diretto" e "indiretto" dei fattori sulle variabili. Nel caso di una rotazione obliqua, le saturazioni fattoriali non coincidono con le correlazioni tra variabili e fattori. Si consideri la figura \@ref(fig:fact-rot5). Nel caso di una rotazione obliqua, la correlazione tra i due fattori comuni viene rappresentata mediante la freccia non direzionata $\phi_{12}$ che collega $\xi_1$ e $\xi_2$. Nel diagramma di percorso della figura successiva ci sono due percorsi legittimi che, in base alle regole di Wright, consentono di collegare ciascuna variabile manifesta a un fattore comune.

Ad esempio, per la variabile $y_1$ e il fattore $\xi_1$, i percorsi sono: la freccia causale $\lambda_{11}$ che rappresenta l'effetto diretto di $\xi_1$ su $y_1$ e il percorso composto che rappresenta l'effetto indiretto di $\xi_1$ su $y_1$, il cui valore numerico è uguale al prodotto $\lambda_{21}\phi_{12}$. Nell'analisi dei percorsi, la correlazione tra $\xi_1$ e $y_1$ è uguale alla somma dei valori numerici dei percorsi legittimi che collegano $y_1$ a $\xi_1$, ovvero $\lambda_{11} + \lambda_{21} \phi_{12}$.

```{figure} images/rot_5.png
---
height: 300px
name: obl-rot-fig
---
Rotazione obliqua.
```

Per illustrare la rotazione obliqua, utilizziamo i dati discussi da
{cite:t}`rencher10methods`. Si consideri la matrice di correlazione presentata qui sotto.

In [ ]:
R <- matrix(
  c(
    1.00,  0.735, 0.711, 0.704,
    0.735, 1.00,  0.693, 0.709,
    0.711, 0.693, 1.00,  0.839,
    0.704, 0.709, 0.839, 1.00
  ),
  ncol = 4,
  byrow = TRUE
)
R


Iniziamo calcolando la soluzione a due fattori mediante il metodo delle
componenti principali e una rotazione Varimax (ovvero, ortogonale). Otteniamo le seguenti saturazioni fattoriali.

In [ ]:
f1_pc <- principal(R, 2, rotate = "varimax") 
f1_pc

Si noti che i due fattori non sono molto distinti. Consideriamo dunque la soluzione  prodotta da una rotazione obliqua. Usiamo qui l'algoritmo Oblimin.

In [ ]:
pr_oblimin <- principal(R, 2, rotate = "oblimin")

La matrice $\hat{\boldsymbol{\Lambda}}$ delle saturazioni fattoriali si ricava come indicato di seguito.

In [ ]:
cbind(pr_oblimin$load[, 1], pr_oblimin$load[, 2])

La matrice $\hat{\boldsymbol{\Phi}}$ di inter-correlazione fattoriale è la seguente.

In [ ]:
pr_oblimin$Phi

La matrice di struttura, che riporta le correlazioni tra indicatori e fattori comuni, si ottiene pre-moltiplicando la matrice $\boldsymbol{\Lambda}$ delle saturazioni fattoriali alla matrice $\boldsymbol{\Phi}$ di inter-correlazione fattoriale.

$$
\text{matrice di struttura} = \boldsymbol{\Lambda}\boldsymbol{\Phi}.
$$

Per esempio, la correlazione tra la prima variabile manifesta e il primo fattore si ottiene nel modo seguente.

In [ ]:
pr_oblimin$load[1, 1] + pr_oblimin$load[1, 2] * pr_oblimin$Phi[2, 1]

L'intera matrice di struttura si può trovare eseguendo la moltiplicazione  $\boldsymbol{\Lambda}\boldsymbol{\Phi}$.

In [ ]:
pr_oblimin$load %*% pr_oblimin$Phi %>% 
  round(3)

## Esempio con `semTools`

Presento qui un esempio di uso di vari metodi di estrazione fattoriale. Tra tali  metodi, la rotazione obliqua Geomin è molto popolare ed è il default di M-Plus. 

Iniziamo a caricare il pacchetto `semTools`.

In [ ]:
suppressPackageStartupMessages(library("semTools")) 

Eseguiamo l'analisi fattoriale esplorativa del classico set di dati di Holzinger e Swineford (1939) il quale è costituito dai punteggi dei test di abilità mentale di bambini di seconda e terza media di due scuole diverse (Pasteur e Grant-White). Nel set di dati originale (disponibile nel pacchetto `MBESS`), sono forniti i punteggi di 26 test. Tuttavia, un sottoinsieme più piccolo con 9 variabili è più ampiamente utilizzato in letteratura. Questi sono i dati qui usati. 

Nel presente esempio, verrà eseguita l'analisi fattoriale esplorativa con l'estrazione di tre fattori. Il metodo di estrazione è `mlr`:

> maximum likelihood estimation with robust (Huber-White) standard errors and a scaled test statistic that is (asymptotically) equal to the Yuan-Bentler test statistic. For both complete and incomplete data.

La soluzione iniziale non è ruotata. 

In [ ]:
unrotated <- efaUnrotate(HolzingerSwineford1939, nf = 3, varList = paste0("x", 1:9), estimator = "mlr")
out <- summary(unrotated)
print(out)


Si noti che, in assenza di rotazione, è impossibile assegnare un significato ai fattori comuni.

### Orthogonal varimax

Utilizziamo ora la rotazione ortogonale Varimax.

In [ ]:
out_varimax <- orthRotate(unrotated, method = "varimax")
out <- summary(out_varimax, sort = FALSE, suppress = 0.3)
print(out)


### Orthogonal Quartimin

Un metodo alternativo per la rotazione ortogonale è Quartimin.

In [ ]:
out_quartimin <- orthRotate(unrotated, method = "quartimin")
out <- summary(out_quartimin, sort = FALSE, suppress = 0.3)
print(out)


### Oblique Quartimin

L'algoritmo Quartimin può anche essere usato per una soluzione obliqua.

In [ ]:
out_oblq <- oblqRotate(unrotated, method = "quartimin")
out <- summary(out_oblq, sort = FALSE, suppress = 0.3)
print(out)


### Orthogonal Geomin

Consideriamo ora la rotazione Geomin. L'algoritmo Geomin fornisce un metodo di rotazione che riduce al minimo la media geometrica delle saturazioni fattoriali innalzate al quadrato. Qui è usato per ottenere una soluzione ortogonale.

In [ ]:
out_geomin_orh <- orthRotate(unrotated, method = "geomin")
out <- summary(out_geomin_orh, sort = FALSE, suppress = 0.3)
print(out)


### Oblique Geomin

La rotazione Geomin può anche essere usata per ottenere una soluzione obliqua.

In [ ]:
out_geomin_obl <- oblqRotate(unrotated, method = "geomin")
out <- summary(out_geomin_obl, sort = FALSE, suppress = 0.3)
print(out)


## Interpretazione 

Per interpretare i fattori comuni latenti, dobbiamo decidere se usare la matrice pattern o la matrice struttura. Un fattore individuato dall'analisi fattoriale è una caratteristica latente univariata che rappresenta l'essenza di un fenomeno psicologico. Dovrebbe essere interpretato come il significato semplice che si trova dietro l'intersezione dei significati delle variabili che saturano nel fattore.

Nella rotazione obliqua, i fattori sono correlati ma vogliamo comunque interpretarli come dimensioni psicologiche distinte. L'etichetta che diamo al fattore $F_1$ dovrebbe aiutare a separare teoricamente il fenomeno psicologico corrispondente a $F_1$ dal fenomeno denotato dall'etichetta del fattore $F_2$, anche se sono correlati.

Se questa è la strategia interpretativa, allora lo strumento principale per l'interpretazione è la matrice pattern. I coefficienti della matrice pattern mostrano l'influenza causale del fattore comune sulle variabili manifeste. La matrice struttura descrive le correlazioni tra variabili e fattori e dipende sia dai percorsi diretti che indiretti. Quindi, non descrive gli effetti diretti dei fattori comuni latenti sulle variabili manifeste ma solo la covariazione tra di loro.
